In [12]:
!pip install boto3
!pip install pandas
!pip install awscli
!pip install webdriver-manager

  Using cached botocore-1.34.84-py3-none-any.whl.metadata (5.7 kB)
Using cached botocore-1.34.84-py3-none-any.whl (12.1 MB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.43
    Uninstalling botocore-1.34.43:
      Successfully uninstalled botocore-1.34.43


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.11.2 requires botocore<1.34.35,>=1.33.2, but you have botocore 1.34.84 which is incompatible.
awscli 1.32.43 requires botocore==1.34.43, but you have botocore 1.34.84 which is incompatible.


  Using cached botocore-1.34.43-py3-none-any.whl.metadata (5.7 kB)
Using cached botocore-1.34.43-py3-none-any.whl (12.0 MB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.84
    Uninstalling botocore-1.34.84:
      Successfully uninstalled botocore-1.34.84


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.11.2 requires botocore<1.34.35,>=1.33.2, but you have botocore 1.34.43 which is incompatible.
boto3 1.34.71 requires botocore<1.35.0,>=1.34.71, but you have botocore 1.34.43 which is incompatible.


In [13]:
# Loading required libraries
import awscli
import boto3
import selenium
import pandas as pd
import time
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait  # Import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [16]:
# Import necessary libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from time import sleep

# Call the webdriver
services = Service(ChromeDriverManager().install())
browser = webdriver.Chrome(service=services)

# Open the URL to scrape
browser.get('https://www.charitiesnys.com/RegistrySearch/search_charities.jsp')

# Locate the input element by XPath
input_element = browser.find_element(By.XPATH,'//*[@id="header"]/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[2]/td[2]/input[1]')
input_element.send_keys('0')

# Click the search button
search_button = browser.find_element(By.XPATH,'//*[@id="header"]/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[10]/td/input[1]')
search_button.click()

# Wait for the page to load
sleep(4)

# Find the table containing the data
table = browser.find_element(By.CSS_SELECTOR, 'table.Bordered')
sleep(1)

# Create an empty list to store data
data = []

# Loop through pages to scrape data
while True:
    # Extract data from each row in the table
    for row in table.find_elements(By.CSS_SELECTOR, 'tr'):
        # Append the text of each cell in the row to the data list
        cols = data.append([cell.text for cell in row.find_elements(By.CSS_SELECTOR, 'td')])

    # Check if there's a next page link
    try:
        # Find and click the link to the next page
        page_link = WebDriverWait(browser, 10).until(
            EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div/table/tbody/tr/td[3]/div/div/span[2]/a[9]'))
        )
        page_link.click()
        
        # Wait for the next page to load
        sleep(4)
        
        # Find the table on the next page
        table = browser.find_element(By.CSS_SELECTOR, 'table.Bordered')
    except:
        # Stop the loop if there's no next page
        break

# Create a DataFrame from the scraped data with column names
df = pd.DataFrame(data, columns=["Organization Name", "NY Reg #", "EIN", "Registrant Type", "City", "State"])

# Drop any rows with missing values
df.dropna(inplace=True)

# Display the DataFrame
df


,Organization Name,NY Reg #,EIN,Registrant Type,City,State
1,"""Forever Captain Poodaman"" The Ahmad Butler Fo...",48-07-16,843800926,NFP,PHILADELPHIA,PA
2,"""Incredibly Blessed"" Inc",49-54-61,842071758,NFP,STATEN ISLAND,NY
3,"""R"" S.U.C.C.E.S.S. Foundation Inc.",49-06-59,874012670,NFP,ROCHESTER,NY
4,"""Studio 5404"" Inc.",44-39-58,463180470,NFP,MASSAPAQUA,NY
5,"""THEY ARE HAITIAN"" FUND, INC.",20-63-46,300170128,NFP,HUDSON,NY
6,"""Y"" Dive, Inc.",48-45-01,854252095,NFP,SAINT ALBANS,NY
7,(ASMA) American Syrian Multicultural Associati...,42-84-63,273130182,NFP,BROOKLYN,NY
8,#FeedHamburg,48-37-35,854150318,NFP,HAMBURG,NY
9,#HicksStrong Inc.,48-10-48,842612081,NFP,CLIFTON PARK,NY
10,#WalkAway Foundation,47-15-80,832820906,NFP,CARLSBAD,CA


In [17]:
import boto3
import pandas as pd

# Specify the name of the S3 bucket
bucket_name = 'database-update-bucket-m10-assn-aniket-tripathi'

# Prepare the CSV file name
filename = 'charities_bureau_scrape_'  # Your group's name
datetime = time.strftime("%Y%m%d%H%M%S")  # Timestamp
s3_key = f"{filename}{datetime}.csv"  # S3 key (filename)

# Convert the DataFrame to CSV string format
csv_buffer = df.to_csv(index=False)

# Upload the CSV file to S3
s3 = boto3.resource('s3')
s3_object = s3.Object(bucket_name, s3_key)
s3_object.put(Body=csv_buffer)

# Print a success message
print("File uploaded successfully to location: s3://{}/{}".format(bucket_name, s3_key))


File uploaded successfully to location: s3://database-update-bucket-m10-assn-aniket-tripathi/charities_bureau_scrape_20240414231647.csv


# End Of File